# Use Python function to predict user's satisfaction for car rental company with `ibm-watson-machine-learning`

This notebook trains a **Keras** (TensorFlow) model to predict customer satisfaction based on the feedback that has been provided. The notebook also demonstrates how you can use the **python function** for the deep learning model data preprocessing required before you start model scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.7 and the watson-machine-learning-client package.


Contents:
1. [Setup](#setup)
2. [Load and explore data](#load)
3. [Create Keras model using TensorFlow backend](#model)
4. [Store the model in the repository](#persistence)
5. [Deploy the model](#deployment)
6. [Score the model](#score)
7. [Define, store and deploy Python function](#ai_function)
8. [Clean up](#cleanup)
9. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [2]:
wml_credentials = {
    "username": username,
    "apikey": api_key,
    "url": url,
    "instance_id": 'openshift',
    "version": '4.0'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [3]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [4]:
client.set.default_space(space_id)

'SUCCESS'

### Install TensorFlow


In [ ]:
!pip install --upgrade tensorflow==2.1 | tail -n 1

**Tip:** Restart the kernel (**Kernel** -> **Restart**)

<a id="load"></a>
## 2. Load and explore data

In this section you read in the `car_rental_training_data.csv` file, load it as a pandas dataFrame and then perform a basic exploration. 


In [ ]:
!pip install wget  | tail -n 1

In [5]:
import wget

link_to_data = 'https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd4.0/data/cars-4-you/car_rental_training_data.csv'
filename = wget.download(link_to_data)

filename

'car_rental_training_data.csv'

Load the data as a pandas dataFrame.


In [6]:
import pandas as pd

data = pd.read_csv(filename, sep=';') 
data.head()


,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,83,Female,M,2,48.85,Inactive,Yes,I thought the representative handled the initi...,0,Product: Availability/Variety/Size,Free Upgrade
1,1307,Female,M,0,55.00,Inactive,No,I have had a few recent rentals that have take...,0,Product: Availability/Variety/Size,Voucher
2,1737,Male,M,0,42.35,Inactive,Yes,car cost more because I didn't pay when I rese...,0,Product: Availability/Variety/Size,Free Upgrade
3,3721,Male,M,2,61.71,Inactive,Yes,I didn't get the car I was told would be avail...,0,Product: Availability/Variety/Size,Free Upgrade
4,11,Male,S,2,56.47,Active,No,If there was not a desired vehicle available t...,1,Product: Availability/Variety/Size,NaN




**Note:** 0 - not satisfied, 1 - satisfied


Extract the required columns and count the number of records.


In [7]:
complain_data = data[['Customer_Service', 'Satisfaction']]

In [8]:
complain_data.count()

Customer_Service    486
Satisfaction        486
dtype: int64

<a id="model"></a>
## 3. Create a Keras model using the TensorFlow backend

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import os
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

### 3.1 Prepare the data


In [10]:
max_fatures = 500

for idx,row in complain_data.iterrows():
    row[0] = row[0].replace('rt',' ')

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(complain_data['Customer_Service'].values)
X = tokenizer.texts_to_sequences(complain_data['Customer_Service'].values)

maxlen = 50

X = pad_sequences(X, maxlen=maxlen)
X.shape

(486, 50)

### Split the data into train and test data sets.


In [11]:
Y = complain_data['Satisfaction'].values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)

X_train.shape,Y_train.shape
X_test.shape,Y_test.shape

((161, 50), (161,))

### 3.2 Design and train the model
Create the network definition based on the Gated Recurrent Unit (Cho et al. 2014).


In [12]:
embedding_vector_length = 32

model = Sequential()
model.add(Embedding(max_fatures, embedding_vector_length, input_length=maxlen))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 32)            16000     
_________________________________________________________________
conv1d (Conv1D)              (None, 50, 32)            3104      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 25, 32)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 72,405
Trainable params: 72,405
Non-trainable params: 0
_________________________________________________________________


### Train the model.


In [14]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=20, batch_size=64)

Train on 325 samples, validate on 161 samples
Epoch 1/20
325/325 [==============================] - 2s 6ms/sample - loss: 0.6900 - accuracy: 0.5354 - val_loss: 0.6869 - val_accuracy: 0.5342
Epoch 2/20
325/325 [==============================] - 0s 392us/sample - loss: 0.6758 - accuracy: 0.5785 - val_loss: 0.6885 - val_accuracy: 0.5342
Epoch 3/20
325/325 [==============================] - 0s 437us/sample - loss: 0.6680 - accuracy: 0.5785 - val_loss: 0.6829 - val_accuracy: 0.5342
Epoch 4/20
325/325 [==============================] - 0s 474us/sample - loss: 0.6463 - accuracy: 0.5785 - val_loss: 0.6630 - val_accuracy: 0.5342
Epoch 5/20
325/325 [==============================] - 0s 434us/sample - loss: 0.6221 - accuracy: 0.5785 - val_loss: 0.6450 - val_accuracy: 0.6149
Epoch 6/20
325/325 [==============================] - 0s 455us/sample - loss: 0.5862 - accuracy: 0.7169 - val_loss: 0.6063 - val_accuracy: 0.6957
Epoch 7/20
325/325 [==============================] - 0s 516us/sample - loss: 0.

In [15]:
("Best accuracy on test: %3.3f" % numpy.array(history.history['val_accuracy']).max())

'Best accuracy on test: 0.957'

**Note:** For purpose of this demo, model tuning has been skipped.

Store and archive the model in the notebook filesystem.



### Evaluate the model

In [16]:
scores = model.evaluate(X_test, Y_test, verbose=0)
("Evaluation Accuracy: %.2f%%" % (scores[1]*100))

'Evaluation Accuracy: 95.65%'

In [17]:
filename = 'complain_model.h5'
model.save(filename)

# Compress Keras model
tar_filename = filename + ".tgz"
cmdstring = "tar -zcvf " + tar_filename + " " + filename
score = tar_filename
os.system(cmdstring);

<a id="persistence"></a>
## 4. Store the model in the repository

In this section, you will learn how to store your model in Watson Machine Learning repository by using the Watson Machine Learning Client.

### 4.1: Publish model
#### Publish model in Watson Machine Learning repository on Cloud.


Get software specification for tensorflow model.

In [18]:
software_spec_id = client.software_specifications.get_id_by_name('tensorflow_2.4-py3.7')
print(software_spec_id)

c4032338-2a40-500a-beef-b01ab2667e27


Store model

In [ ]:

metadata = {
    client.repository.ModelMetaNames.NAME: "CARS4U - Satisfaction Prediction Model",
    client.repository.ModelMetaNames.TYPE: 'tensorflow_2.4',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID : software_spec_id
}
print(metadata)    
stored_model_details = client.repository.store_model(tar_filename, meta_props=metadata)

### 4.2: Get model details

In [ ]:
import json

model_uid = client.repository.get_model_uid(stored_model_details)

model_details = client.repository.get_details(model_uid)
print(json.dumps(model_details, indent=2))

<a id="deployment"></a>
## 5. Deploy the model
In this section you will learn how to create batch deployment to create job using the Watson Machine Learning Client.

You can use commands below to create batch deployment for stored model (web service).

### 5.1: Create model deployment


In [21]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "CARS4U - Satisfaction Prediction Model Deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [22]:
deployment_details = client.deployments.create(model_uid, meta_props, background_mode=False)




#######################################################################################

Synchronous deployment creation for uid: '839f757e-a9a0-4da4-901a-61a17ad9db76' started

#######################################################################################


initializing.
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='7445e109-6b0f-4877-8043-adb22aaddd7f'
------------------------------------------------------------------------------------------------




**Note**: Here we use deployment url saved in published_model object. In next section, we show how to retrieve deployment url from Watson Machine Learning instance.



In [23]:
deployment_uid = client.deployments.get_uid(deployment_details)

Now, You can list all deployments.

In [ ]:
client.deployments.list()

### 5.2 Get deployment details

In [24]:
client.deployments.get_details(deployment_uid)

{'entity': {'asset': {'id': '839f757e-a9a0-4da4-901a-61a17ad9db76'},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'XXS', 'num_nodes': 1},
  'name': 'CARS4U - Satisfaction Prediction Model Deployment',
  'online': {},
  'space_id': '83b00166-9047-4159-b777-83dcb498e7ab',
  'status': {'online_url': {'url': 'https://wmlgmc-cpd-wmlgmc.apps.wmlautoai.cp.fyre.ibm.com/ml/v4/deployments/7445e109-6b0f-4877-8043-adb22aaddd7f/predictions'},
   'state': 'ready'}},
 'metadata': {'created_at': '2020-12-08T12:36:40.361Z',
  'id': '7445e109-6b0f-4877-8043-adb22aaddd7f',
  'modified_at': '2020-12-08T12:36:40.361Z',
  'name': 'CARS4U - Satisfaction Prediction Model Deployment',
  'owner': '1000330999',
  'space_id': '83b00166-9047-4159-b777-83dcb498e7ab'}}

<a id="score"></a>
## 6. Score the model
Let's see if our deployment works.


You can use below method to do test scoring request against deployed model.



In [25]:
deployment_id = client.deployments.get_id(deployment_details)

In [26]:
deployment_id

'7445e109-6b0f-4877-8043-adb22aaddd7f'

**Action**: Prepare scoring payload with records to score.

In [27]:
index = 5

scoring_data = X[index].tolist()
print(X_test[index])
print(Y_test[index])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   4  43  19   3 242  25
 235   6  22 353   1 237  12  10   8 107  14  34   5  56]
1


In [28]:
scoring_payload = {client.deployments.ScoringMetaNames.INPUT_DATA:
    [
        {
        'values': [scoring_data]
        }
   ]
}

In [29]:
scores = client.deployments.score(deployment_id, scoring_payload)

Let's print the scoring results.

In [30]:
print(str(scores))

{'predictions': [{'id': 'dense', 'fields': ['prediction', 'prediction_classes', 'probability'], 'values': [[[0.0005826586857438087], [0], [0.0005826586857438087]]]}]}


<a id="ai_function"></a>
## 7. Define, store and deploy Python function
Let's define a function that does data preprocessing and model scoring. As we saw in the previous cells, the model expects numerical input, so the text comment needs to be preprocessed.

### 7.1 Define generic parameters

In [31]:
ai_params = {
    'deployment_id': deployment_id,
    'wml_credentials': wml_credentials,
    'word_index': tokenizer.word_index,
    'space_id': space_id
}

In [32]:
def score_generator(params=ai_params):
    import re
    from ibm_watson_machine_learning import APIClient
    client = APIClient(params['wml_credentials'])
    client.set.default_space(params['space_id'])
    def score(payload):
        max_fatures = 500
        maxlen = 50
        preprocessed_records = []
        complain_data = payload['input_data'][0]['values']
        word_index = params['word_index']
        for data in complain_data:
            comment = data[0]
            cleanString = re.sub(r"[!\"#$%&()*+,-./:;<=>?@[\]^_`{|}~]", "", comment)
            splitted_comment = cleanString.split()[:maxlen]
            hashed_tokens = []
            for token in splitted_comment:
                index = word_index.get(token, 0)
                if index < 501 and index > 0:
                    hashed_tokens.append(index)
            hashed_tokens_size = len(hashed_tokens)
            padded_tokens = [0]*(maxlen-hashed_tokens_size) + hashed_tokens
            preprocessed_records.append(padded_tokens)
        scoring_payload = {'values': preprocessed_records}
        scoring_payload = {client.deployments.ScoringMetaNames.INPUT_DATA: [scoring_payload]}
        return client.deployments.score(params['deployment_id'], scoring_payload)
    return score


#### Test the function locally


In [33]:
sample_payload = {
    "input_data": [
        {
            "fields": ["feedback"],
            "values": [['delayed shuttle, almost missed flight, bad customer service'],['The car was great and they were able to provide all features I wanted with limited time they had.']]
        }       
    ]
}

In [34]:
print(sample_payload)

{'input_data': [{'fields': ['feedback'], 'values': [['delayed shuttle, almost missed flight, bad customer service'], ['The car was great and they were able to provide all features I wanted with limited time they had.']]}]}


In [35]:
score = score_generator()
score_ai = score(sample_payload)
print(score_ai)

{'predictions': [{'id': 'dense', 'fields': ['prediction', 'prediction_classes', 'probability'], 'values': [[[0.005412891041487455], [0], [0.005412891041487455]], [[0.9990930557250977], [1], [0.9990930557250977]]]}]}


**Note:** 0 - not satisfied. 1 - satisfied

### 7.2 Store the function

In [36]:
software_spec_id = client.software_specifications.get_id_by_name("default_py3.7")
software_spec_id 

'e4429883-c883-42b6-87a8-f419d64088cd'

In [37]:
meta_data = {
    client.repository.FunctionMetaNames.NAME: 'CARS4U - Satisfaction Prediction - AI Function',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: software_spec_id,
}

function_details = client.repository.store_function(score_generator, meta_data)

In [ ]:
client.repository.list_functions()

### 7.3 Deploy the function

In [39]:
function_uid = client.repository.get_function_uid(function_details)
published_function_id = client.repository.get_function_id(function_details)
client.repository.get_function_details(function_uid)

{'entity': {'software_spec': {'id': 'e4429883-c883-42b6-87a8-f419d64088cd',
   'name': 'default_py3.7'},
  'type': 'python'},
 'metadata': {'created_at': '2020-12-08T12:37:11.870Z',
  'id': '89a291fa-3268-498d-a034-d9f6379b80ea',
  'modified_at': '2020-12-08T12:37:14.569Z',
  'name': 'CARS4U - Satisfaction Prediction - AI Function',
  'owner': '1000330999',
  'space_id': '83b00166-9047-4159-b777-83dcb498e7ab'},
 'system': {'warnings': []}}

In [40]:
meta_data = {
    client.deployments.ConfigurationMetaNames.NAME: 'CARS4U - Satisfaction Prediction - AI Function Deployment',
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [41]:
function_deployment_details = client.deployments.create(published_function_id,meta_data)



#######################################################################################

Synchronous deployment creation for uid: '89a291fa-3268-498d-a034-d9f6379b80ea' started

#######################################################################################


initializing...
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='913d9d94-339c-4b31-95e1-d527d6edcae7'
------------------------------------------------------------------------------------------------




### 7.4 Score function

In [42]:
import json
print(function_deployment_details['metadata']['id'])
scoring_response = client.deployments.score(function_deployment_details['metadata']['id'], sample_payload)

print(json.dumps(scoring_response, indent=3))

913d9d94-339c-4b31-95e1-d527d6edcae7
{
   "predictions": [
      {
         "id": "dense",
         "fields": [
            "prediction",
            "prediction_classes",
            "probability"
         ],
         "values": [
            [
               [
                  0.005412891041487455
               ],
               [
                  0
               ],
               [
                  0.005412891041487455
               ]
            ],
            [
               [
                  0.9990930557250977
               ],
               [
                  1
               ],
               [
                  0.9990930557250977
               ]
            ]
         ]
      }
   ]
}


<a id="cleanup"></a>
## 8. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 9. Summary and next steps

You successfully completed this notebook! You learned how to use Keras machine learning library as well as Watson Machine Learning for model creation and deployment.

Check out our [Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/analyze-data/wml-setup.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Wojciech Jargielo**, Software Engineer

Copyright © 2020, 2021 IBM. This notebook and its source code are released under the terms of the MIT License.

